# Polars
> DataFrames increíblemente rápidos en Rust, Python y Node.js.

- toc: true 
- badges: false
- comments: true
- categories: [python]
- image: images/polars.PNG

# Introducción

[Polars](https://github.com/pola-rs/polars) es una librería de DataFrames increíblemente rápida implementada en [Rust](https://github.com/rust-lang/rust) utilizando [Arrow Columnar Format](https://arrow.apache.org/docs/format/Columnar.html) de Apache como modelo de memoria.


* Lazy | eager execution
* Multi-threaded
* SIMD (Single Instruction, Multiple Data)
* Query optimization
* Powerful expression API
* Rust | Python | ...

Esta sección tiene como objetivos presentarle *Polars* a través de ejemplos y comparándolo con otras soluciones.

> **Nota**: Si usted no esta familiarizado con la manipulación de datos en Python, se recomienda partir leyendo sobre la librería de [Pandas](https://pandas.pydata.org/docs/). También, se deja como referencia el curso de [Manipulación de Datos](https://gitlab.com/FAAM/python_data_manipulation).

# Primeros Pasos

## Instalación

Para instalar **Polars**, necesitará usar la línea de comando. Si ha instalado Anaconda, puede usar:

```
conda install -c conda-forge polars
```

De lo contrario, puede instalar con pip:


```
pip install polars
```

> **Nota**: Todos los binarios están preconstruidos para Python v3.6+.


# Rendimiento
*Polars* es muy rápido y, de hecho, es una de las mejores soluciones disponibles. Tomemos como referencia [db-benchmark](https://h2oai.github.io/db-benchmark/) de h2oai. Esta página tiene como objetivo comparar varias herramientas similares a bases de datos populares en la ciencia de datos de código abierto. Se ejecuta regularmente con las últimas versiones de estos paquetes y se actualiza automáticamente.

También se incluye la sintaxis que se cronometra junto con el tiempo. De esta manera, puede ver de inmediato si está realizando estas tareas o no, y si las diferencias de tiempo le importan o no. Una diferencia de 10x puede ser irrelevante si eso es solo 1s frente a 0,1s en el tamaño de sus datos.

A modo de ejemplo, veamos algunos ejemplos de *performances* de distintas librerías para ejecutar distintos tipos de tareas sobre datasets con distintos tamaños. Para el caso de tareas básicas sobre un dataset de **50 GB**, *Polars* supera a librerías espacializadas en distribución de Dataframes como *Spark* (143 segundos vs 568 segundos). Por otro lado, librerías conocidas en Python como *Pandas* o *Dask* se tiene el problema de **out of memory**.

![](../images/polars/db-benchmark.png)

# Expresiones en Polars

Polars tiene un poderoso concepto llamado expresiones. Las expresiones polares se pueden usar en varios contextos y son un mapeo funcional de `Fn(Series) -> Series`, lo que significa que tienen `Series` como entrada y `Series` como salida. Al observar esta definición funcional, podemos ver que la salida de un `Expr` también puede servir como entrada de un `Expr`.

Eso puede sonar un poco extraño, así que vamos a dar un ejemplo.

La siguiente es una expresión:

```python
pl.col("foo").sort().head(2)
```

El fragmento anterior dice seleccionar la columna "foo", luego ordenar esta columna y luego tomar los primeros 2 valores de la salida ordenada. El poder de las expresiones es que cada expresión produce una nueva expresión y que se pueden canalizar juntas. Puede ejecutar una expresión pasándola en uno de los contextos de ejecución polares. Aquí ejecutamos dos expresiones ejecutando df.select:

```python
df.select([
     pl.col("foo").sort().head(2),
     pl.col("barra").filter(pl.col("foo") == 1).sum()
])
```

Todas las expresiones se ejecutan en paralelo. (Tenga en cuenta que dentro de una expresión puede haber más paralelización).

## Expresiones

En esta sección veremos algunos ejemplos, pero primero vamos a crear un conjunto de datos:

In [2]:
import polars as pl
import numpy as np

In [3]:
np.random.seed(12)

df = pl.DataFrame(
    {
        "nrs": [1, 2, 3, None, 5],
        "names": ["foo", "ham", "spam", "egg", None],
        "random": np.random.rand(5),
        "groups": ["A", "A", "B", "C", "B"],
    }
)
print(df)

shape: (5, 4)
┌──────┬───────┬──────────┬────────┐
│ nrs  ┆ names ┆ random   ┆ groups │
│ ---  ┆ ---   ┆ ---      ┆ ---    │
│ i64  ┆ str   ┆ f64      ┆ str    │
╞══════╪═══════╪══════════╪════════╡
│ 1    ┆ foo   ┆ 0.154163 ┆ A      │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ 2    ┆ ham   ┆ 0.74     ┆ A      │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ 3    ┆ spam  ┆ 0.263315 ┆ B      │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ null ┆ egg   ┆ 0.533739 ┆ C      │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ 5    ┆ null  ┆ 0.014575 ┆ B      │
└──────┴───────┴──────────┴────────┘


Puedes hacer mucho con las expresiones. Son tan expresivos que a veces tienes varias formas de obtener los mismos resultados. Para tener una idea de ellos, veamos algunos ejemplos.